In [1]:
import pandas as pd
import re
import numpy
import pickle

In [2]:
df =  pd.read_csv('base_data/pride_table.csv').astype(str)

In [3]:
df.head(2)

,dataset_id,sample_protocol,data_protocol,description,instruments,exp_types,quant_methods,labhead_fullname
0,PXD000001,Not available,Two extra files have been added post-publicati...,Expected reporter ion ratios: Erwinia peptides...,"LTQ Orbitrap Velos, instrument model",Bottom-up proteomics,nan,nan
1,PXD000002,Not available,Not available,Human saliva samples from a 36 year old male w...,"instrument model, LTQ Velos",Bottom-up proteomics,nan,nan


In [4]:
method = df['sample_protocol'][(df['sample_protocol'] != 'Not available') & (df['sample_protocol'] != 'nan')].reset_index(drop=True)

In [5]:
method.iloc[3]

'Conditioned media preparation and HPLC-isobaric tags for relative and absolute quantitation mass-spectrometry (iTRAQ MS). Twenty million cells were transfected with either control or Gαs siRNA, then were seeded in 100-mm cell culture plates in DMEM + 10% FBS. Two days later, cells were washed twice with pre-warmed PBS. Conditioned media was harvested another 24 h later, and centrifuged at 20,000 g for 10 min at 4°C to remove cell debris. To enrich for secreted proteins in the conditioned media, conditioned media samples were centrifuged using Amicon centrifugal filters with a 3kDa cutoff (Millipore, Billerica, MA). A total of 8 concentrated conditioned media samples were independently prepared: 4 samples from control siRNA-treated cells, and 4 samples from Gαs siRNA-treated cells. Samples were stored at –70°C until they were analyzed by iTRAQ labeling and LC-MS/MS. iTRAQ labeling – Total protein concentration of each sample was measured using the Bradford assay. A quantity of 100 µg w

In [7]:
# finding concentrations
hyphen_words = r'\w+-\w+(?:-\w)*'
text = method.iloc[3]
res = [s for s in re.findall(hyphen_words, text)]
for conc in res:
    print(conc)

HPLC-isobaric
mass-spectrometry
100-mm
pre-warmed
siRNA-treated
siRNA-treated
LC-MS
2-carboxyethyl
mg-C18
12-well
12-cm
pH-gradient
LC-MALDI-T
in-house
on-line
α-Cyano-4-h
time-of-f
time-of-f
Batch-mode
signal-to-n
VGF-NERP1
LTQ-Orbitrap


In [8]:
def extract_hyphen_words(text):
    hyphen_words = r'\w+-\w+(?:-\w)*\b'
    
    if type(text) == str:
        hyphen_words = [s.lower() for s in re.findall(hyphen_words, text)]
        return hyphen_words
    else:
        return []

In [9]:
method = method.to_frame()

In [10]:
method['hyphen_words'] = method['sample_protocol'].apply(lambda x: extract_hyphen_words(x))

In [11]:
method.head(20)

,sample_protocol,hyphen_words
0,The crude membranes from 5 P56-P70 Glun1TAP/TA...,"[p56-p70, re-suspended, immuno-capture, peptid..."
1,"Breast cancer tissue lysates, reduction, alkyl...","[methyl-methanethiosulfonate, 8-plex, lc-ms]"
2,In-gel digests were performed as described in ...,"[in-gel, sds-page, nano-esi, lc-ms, on-line]"
3,Conditioned media preparation and HPLC-isobari...,"[hplc-isobaric, mass-spectrometry, 100-mm, pre..."
4,Proteins were eluted from washed beads by addi...,"[2-fold, sds-gel, bis-tris, chromatography-tan..."
5,Exosomes were isolated from 10 healthy volunte...,"[j26-xp, ja-17, 50-ml, 22-µm, l-100, fixed-ang..."
6,Low resolution mass spectrometry: For the shot...,"[1d-lc, 1d-lc, 2d-lc]"
7,"Arabidopsis plants, Wassilewskija background (...","[12-h, percoll-purified, sds-page, in-gel, lc-..."
8,Zebrafish larvae were ground with a pestle in ...,"[sigma-aldrich, 10-lane, 4-12, bis-tris, mini-..."
9,Affinity purification and nano-liquid chromato...,"[nano-liquid, sirt7-interacting, co-immunoprec..."


In [12]:
method.iloc[13, 0]

'Whole Cell Lysis: 109 cells were lysed in 50 ml lysis buffer (8M urea, 0.1% RapiGest, 0.1M ammoniumbicarbonate) and disrupted by two cycles of sonication for 20 seconds (Hielscher Ultrasonicator). Protein concentration was determined by BCA assay (Thermo Fisher Scientific) using a small sample aliquot. Lys-C/Trypsin Protein Digestion: Lysates were reduced with 5 mM TCEP for 60 min at 37°C, alkylated with 10 mM iodoacetamide for 30 min in the dark and quenched with 12.5 mM N-acetyl-cysteine. Proteins were digested by addition of Lys-C (Wako) for 4 h at 37°C (protein to Lys-C ration 100:1), diluted to a final urea concentration of 1.5 M with 100 mM ammoniumbicarbonate buffer and further digested by addition of trypsin (Promega) and incubation at 37°C for more than 15 h (protein to trypsin ratio: 50:1). After digestion, the samples were supplemented with TFA to a final concentration of 0.5% and HCl to a final concentration of 50 mM. Peptides were desalted on C18 reversed phase spin colum